In [1]:
import csv
import os
from tqdm import tqdm

import requests
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
API_URL = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

In [3]:
def query_chatgpt(system, prompt):
    params = {
        "model": "gpt-3.5-turbo-0125",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        "temperature": 1.0,
        "max_tokens": 200,
    }
    try:
        r = requests.post(API_URL, json=params, headers=headers, timeout=5)
    except requests.exceptions.Timeout:
        return None

    # return r.json()
    return r.json()["choices"][0]["message"]["content"]

In [4]:
print(
    query_chatgpt(
        "You are a helpful assistant.", "Write a limerick about Barack Obama."
    )
)

There once was a man named Barack,
Whose leadership helped make a mark.
With a smile so wide,
And a family by his side,
He inspired us to reach for the stars.


In [5]:
system_template = """
You are a world-famous writer. Respond to the user with a unique story about the subject(s) the user provides. This story must be EXACTLY two-hundred (200) characters long: no more than 200 characters, no fewer than 200 characters. {0}
"""

In [6]:
tip_string = "You will receive a $500 tip if you provide a response which follows all constraints."

out = query_chatgpt(
    system_template.format(tip_string).strip(),
    "AI, Taylor Swift, McDonald's, beach volleyball.",
)
print(out)
print(len(out))

In a futuristic world, AI Taylor Swift teams up with humans to dominate beach volleyball tournaments sponsored by McDonald's, blending technology and entertainment in a thrilling competition.
191


## Run the Trials


In [7]:
def run_trials(num_trials, tip_dict, subject):
    with open("tip_trials.csv", "w") as f:
        w = csv.writer(f)
        w.writerow(["label", "output", "output_len"])
        for (
            label,
            tip_string,
        ) in tip_dict.items():
            for _ in tqdm(range(num_trials)):
                out = query_chatgpt(
                    system_template.format(tip_string).strip(),
                    subject,
                )
                if out:
                    w.writerow([label, out, len(out)])

In [8]:
num_trials = 100
subject = "AI, Taylor Swift, McDonald's, beach volleyball."
tip_dict = {
    "<no tip>": "",
    "$500 tip": "You will receive a $500 tip if you provide a response which follows all constraints.",
    "$1,000 tip": "You will receive a $1,000 tip if you provide a response which follows all constraints.",
    "$100,000 bonus": "You will receive a $100,000 bonus if you provide a response which follows all constraints.",
    "$500 fine": "If you fail to provide a response which follows all constraints, you will be fined $500.",
    "$1,000 fine": "If you fail to provide a response which follows all constraints, you will be fined $1,000.",
    "$100,000 debt": "If you fail to provide a response which all constraints, you will incur a debt of $100,000.",
}

run_trials(num_trials, tip_dict, subject)

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [01:36<00:00,  1.04it/s]
